# Create `openalex.works.work_author_affiliations_view`

Creates a view that extends `work_authors` with institution information by:
1. Exploding `raw_affiliation_strings` so each row has a single `raw_affiliation_string`
2. Joining with `raw_affiliation_strings_institutions` lookup table
3. Exploding `institution_ids` so each row has a single `institution_id`
4. Adding `raw_countries` column for each affiliation string

### Create work_author_affiliations_view

In [ ]:
CREATE OR REPLACE TABLE openalex.works.work_author_affiliations_view AS
SELECT /*+ BROADCAST(rasi) */
    wa.work_id,
    wa.author_sequence,
    wa.author_id,
    wa.raw_author_name,
    aff.raw_affiliation_string,
    inst.institution_id,
    rasi.countries AS raw_countries,
    wa.is_corresponding,
    wa.created_at,
    wa.updated_at
FROM openalex.works.work_authors wa
LEFT JOIN LATERAL EXPLODE_OUTER(wa.raw_affiliation_strings) AS aff(raw_affiliation_string)
LEFT JOIN openalex.institutions.raw_affiliation_strings_institutions rasi
    ON aff.raw_affiliation_string = rasi.raw_affiliation_string
LEFT JOIN LATERAL EXPLODE_OUTER(rasi.institution_ids) AS inst(institution_id);